In [1]:
from scrapy import signals
from scrapy.utils.project import get_project_settings
from scrapy.crawler import CrawlerProcess
from scrapy.xlib.pydispatch import dispatcher
from multiprocessing.queues import Queue
import scrapy
import multiprocessing
import datetime
import collections
import urllib
from scrapy import Request as Request

/home/jun_gentoo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: ScrapyDeprecationWarning: Importing from scrapy.xlib.pydispatch is deprecated and will no longer be supported in future Scrapy versions. If you just want to connect signals use the from_crawler class method, otherwise import pydispatch directly if needed. See: https://github.com/scrapy/scrapy/issues/1762


In [2]:
class CrawlerWorker(multiprocessing.Process):

    def __init__(self, spider, result_queue):
        multiprocessing.Process.__init__(self)
        self.result_queue = result_queue

        self.crawler = CrawlerProcess(get_project_settings())
        #if not hasattr(project, 'crawler'):
        #    self.crawler.install()
        #self.crawler.configure()

        self.items = []
        self.spider = spider
        dispatcher.connect(self._item_passed, signals.item_passed)

    def _item_passed(self, item):
        self.items.append(item)

    def run(self):
        self.crawler.crawl(self.spider)
        self.crawler.start()
        self.crawler.stop()
        self.result_queue.put(self.items)

In [11]:
class CanberraBusinessSpider(scrapy.Spider):
    name = "CanberraBusinessSpider"
    allowed_domains = ["www.seekbusiness.com.au"]
    start_urls = ['https://www.seekbusiness.com.au/businesses-for-sale/in-canberra-act-2601?rad=50']

    def parse(self, response):
        next_page=response.xpath('//a[contains(@class,"next")]/@href').extract()[0]
        next_page_link=urllib.basejoin(response.url,next_page)
        #print(next_page_link)
        #yield Request(next_page_link,callback=self.parse)
        business_tags=response.xpath('//a[contains(@href,"business-listing") and count(@*)=1]')
        for invidual_business in business_tags:
            title=invidual_business.xpath('text()').extract()[0]
            link=invidual_business.xpath('@href').extract()[0]
            full_link=urllib.basejoin(response.url,link)
            business={'title':title,'link':full_link}
            print(business)
        #print(business_tags)

In [12]:
def main():
    result_queue = Queue()
    crawler = CrawlerWorker(CanberraBusinessSpider(), result_queue)
    crawler.start()
    for item in result_queue.get():
        print item
        
if __name__=="__main__":
    main()

2017-01-17 16:37:47 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-17 16:37:47 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-17 16:37:47 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-17 16:37:47 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-17 16:37:47 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-17 16:37:47 [scrapy] INFO: Overridden settings: {}
2017-01-17 16:37:47 [scrapy] INFO: Overridden settings: {}
2017-01-17 16:37:47 [scrapy] INFO: Overridden settings: {}
2017-01-17 16:37:47 [scrapy] INFO: Overridden settings: {}
2017-01-17 16:37:47 [scrapy] INFO: Overridden settings: {}
2017-01-17 16:37:47 [scrapy] INFO: Enabled extensions:
['scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2017-01-17 16:37:47 [scrapy] INFO: Enabled extensions:
['scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.co

{'link': u'https://www.seekbusiness.com.au/business-listing/mos-mobiles-is-looking-for-licensed-dealers-to-operate-existing-vodafone-stores/281849?s=3001586', 'title': u"Mo's Mobiles is looking for Licensed Dealers to operate existing Vodafone stores"}
{'link': u'https://www.seekbusiness.com.au/business-listing/gozleme-king-franchise-turkish-street-food-cafe/270622?s=3001586', 'title': u'Gozleme King Franchise : Turkish Street Food & Cafe'}
{'link': u'https://www.seekbusiness.com.au/business-listing/healthy-start-civic/258794?s=3001586', 'title': u'Healthy Start Civic'}
{'link': u'https://www.seekbusiness.com.au/business-listing/the-fat-goanna-cafe/290623?s=3001586', 'title': u'The Fat Goanna Cafe'}
{'link': u'https://www.seekbusiness.com.au/business-listing/community-sector-rto-for-sale-delegate-authority-funding-2-states-excellent/286398?s=3001586', 'title': u'Community sector RTO for sale. Delegate Authority. Funding- 2 states. Excellent!'}
{'link': u'https://www.seekbusiness.com.au

2017-01-17 16:37:49 [scrapy] INFO: Closing spider (finished)
2017-01-17 16:37:49 [scrapy] INFO: Closing spider (finished)
2017-01-17 16:37:49 [scrapy] INFO: Closing spider (finished)
2017-01-17 16:37:49 [scrapy] INFO: Closing spider (finished)
2017-01-17 16:37:49 [scrapy] INFO: Closing spider (finished)
2017-01-17 16:37:50 [scrapy] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 268,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET': 1,
 'downloader/response_bytes': 28938,
 'downloader/response_count': 1,
 'downloader/response_status_count/200': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2017, 1, 17, 5, 37, 50, 29701),
 'log_count/DEBUG': 2,
 'log_count/INFO': 7,
 'response_received_count': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2017, 1, 17, 5, 37, 47, 833044)}
2017-01-17 16:37:50 [scrapy] INFO: Dumping Scrapy stats:
{'